# Text Processing

### setup

##### install

In [ ]:
!pip install -q transformers datasets
!pip install -q wandb

# navec embeddings
!pip install -q navec
!wget https://storage.yandexcloud.net/natasha-navec/packs/navec_hudlit_v1_12B_500K_300d_100q.tar

In [ ]:
# repo
!git config --global user.email "SECRET_NAME@mail.ru"
!git config --global user.name "SECRET NAME"
!git clone https://github.com/Abletobetable/smart-product.git

##### import

In [ ]:
import os
import re
import json
import shutil
import pandas as pd
import numpy as np

import torch
from transformers import AutoModel, AutoTokenizer
from transformers import AutoModelForSequenceClassification

# init run for efficient logging
import wandb
run = wandb.init(project="kazan_internship2023")

# load navec pretrained embeddings
from navec import Navec
path = 'navec_hudlit_v1_12B_500K_300d_100q.tar'
navec = Navec.load(path)

# imports from repo
%cd /content/smart-product
from src.text_features import create_average_navec_embed, \
                              preprocess_attributes, \
                              filter_description, \
                              concatenate_text_fields, \
                              filter_characteristics
%cd /content

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(DEVICE)

### text processing

##### load preprocessed dataset

In [4]:
# directory with saved artifact
table_dataset_version = 'processed_table:v0'

In [ ]:
artifact = run.use_artifact(f'abletobetable/kazan_internship2023/{table_dataset_version}', 
                            type='preprocessed_data')
artifact_dir = artifact.download()

In [6]:
train_df = pd.read_csv('/content/artifacts/processed_table:v0/train_processed.tsv', 
                       sep='\t')
predict_df = pd.read_csv('/content/artifacts/processed_table:v0/predict_processed.tsv', 
                         sep='\t')

In [ ]:
train_df.columns

#### work with data

##### informative text fields (title, description and attributes)

processing

In [ ]:
# title
prep_title_train = train_df['title']
prep_title_predict = predict_df['title']

# attributes
prep_attrib_train = preprocess_attributes(train_df['attributes'])
prep_attrib_predict = preprocess_attributes(predict_df['attributes'])

# description
prep_descrip_train = filter_description(train_df['description'])
prep_descrip_predict = filter_description(predict_df['description'])

# concatenate
concat_text_fields_train = concatenate_text_fields(train_df[['category_id', 
                                                             'product_id']], 
                                                   prep_title_train, 
                                                   prep_attrib_train, 
                                                   prep_descrip_train)

concat_text_fields_predict = concatenate_text_fields(predict_df['product_id'], 
                                                     prep_title_predict, 
                                                     prep_attrib_predict, 
                                                     prep_descrip_predict)

100%|██████████| 16860/16860 [00:00<00:00, 64112.51it/s]


save dataset

In [ ]:
# will save output in this folder
folder_path = '/content/text_fields_dataset'
if not os.path.exists(folder_path):
    os.mkdir(folder_path)


concat_text_fields_train.to_csv(f'{folder_path}/text_fields_train.tsv', 
                                  sep='\t', index=False)

concat_text_fields_predict.to_csv(f'{folder_path}/text_fields_predict.tsv', 
                                  sep='\t', index=False)

log processed text dataset

In [ ]:
text_dataset = wandb.Artifact("text_fields_dataset", type="preprocessed_data")
text_dataset.add_dir(folder_path)
run.log_artifact(text_dataset)

wandb: Adding directory to artifact (/content/text_fields_dataset)... Done. 0.9s


##### other information

every field (if it not empty) is dict
so I will get every key from fields and concatenate in one string

In [54]:
dict_text_train = pd.DataFrame(filter_characteristics(train_df))
dict_text_train['category_id'] = train_df['category_id']
dict_text_train['product_id'] = train_df['product_id']

dict_text_predict = pd.DataFrame(filter_characteristics(predict_df))
dict_text_predict['product_id'] = predict_df['product_id']

# will save output in this folder
folder_path = '/content/dict_text_dataset'
if not os.path.exists(folder_path):
    os.mkdir(folder_path)

dict_text_train.to_csv(f'{folder_path}/dict_text_train.tsv', 
                         sep='\t', index=False)

dict_text_predict.to_csv(f'{folder_path}/dict_text_predict.tsv', 
                         sep='\t', index=False)

100%|██████████| 16860/16860 [00:00<00:00, 41482.43it/s]


log processed text dataset

In [55]:
text_dataset = wandb.Artifact("dict_text_dataset", type="preprocessed_data")
text_dataset.add_dir(folder_path)
run.log_artifact(text_dataset)

wandb: Adding directory to artifact (/content/dict_text_dataset)... Done. 0.1s
